In [1]:
%matplotlib widget

import sys  
sys.path.insert(0, '/home/cbisot/pycode/MscThesis/')
import pandas as pd
from sample.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from sample.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from sample.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from sample.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from sample.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from sample.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from sample.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from sample.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from sample.notebooks.analysis.data_info import *
inst = (19,0,25)

exp = get_exp(inst,directory_project)
def criter(max_growth,length):
    return(a*length+b*max_growth>=2.5 and max_growth>=50)
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
hyph_anas_tip_hyph = [hyphat for hyphat in exp.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
branches = {}
lengths = {}
bd = []
ts = []
rh_ends = [rh.end for rh in RH]
daughters = {}
begins = {}
for rh in RH:
    tp1 = rh.ts[-1]
    t = rh.ts[0]
    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
    curv_length = np.sum([get_length_um(seg) for seg in pixels])
    branch = []
    daughter_list=[]
    begins[rh] = nodes[1]
    for node in nodes[1:-1]:
        if node not in [hyph.end for hyph in hyph_anas_tip_hyph]:
            node_obj  = node
#             t0 = node_obj.ts()[0]
#             neighbours = node_obj.neighbours(t0)
#             if t0 in rh.ts:
#                 nodes_hyph,edge = rh.get_nodes_within(t0)
#                 for neighbour in neighbours:
#                     if neighbour.label not in nodes_hyph:
#                         daughter_list.append(neighbour)
#                         if neighbour in rh_ends and neighbour!=rh.end:
            branch.append(node)    
    bd.append(len(branch)/curv_length)
    ts.append(get_time(exp,0,t))
    branches[rh] = branch
    daughters[rh] = daughter_list
    lengths[rh] = curv_length

In [27]:
data = pd.DataFrame(list(zip(ts,bd)),columns = ['time','branchin_dist'])
corrected = data.loc[(data['branchin_dist']>0)]
corrected['real_branchin_dist'] = 1/corrected['branchin_dist']

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
plt.close('all')
abcisse = 'time'
ordinate = 'real_branchin_dist'
tab = corrected
baits_sort = tab.sort_values(abcisse)
N=100
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')

ax.set_ylabel('Branching distance (BAS and RH undistinguished) ($\mu m$)',color='red')
# ax.set_xlim(0,70)
ax.set_ylim(0,1500)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))

ax.scatter(tab[abcisse],tab[ordinate],alpha=0.3,color='red')
ax.plot(moving_av[abcisse],moving_av[ordinate],color='red',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.spines['right'].set_color('red')
ax.set_title((inst,np.mean(corrected['real_branchin_dist'])))

ax.tick_params(axis='y', colors='red')  #setting up Y-axis tick color to black

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [ ]:
np.mean(corrected['real_branchin_dist'])

In [29]:
exp.plot_raw(-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# bd = [790,983,1078,664,1593,860,1132,809,607,573,1455,672,754]
# nconnection = [11,4,3,8,2,14,3,6,10,4,3,5,5]

# total_network = [3589,1668,1301,1187,348,2776,528,2413,2920,1034,1557,1795]

In [2]:
bd = [790,983,1078,664,860,1132,809,607,573,1455,672,754,1136]
nconnection = [11,4,3,8,14,3,6,10,4,3,5,5,2]

total_network = [3589,1668,1301,1187,2776,528,2413,2920,1034,1557,1180,1795,1497]

In [3]:
data = pd.DataFrame(list(zip(nconnection,bd,total_network)),columns = ['n_connect','branchin_dist','total_network'])
corrected = data.loc[(data['branchin_dist']>0)]
corrected['length_connection_rat'] = corrected['total_network']/corrected['n_connect']

In [11]:
#### from scipy.optimize import curve_fit
plt.rcParams.update({
    "font.family": "verdana",
'font.weight' : 'normal',
'font.size': 20})
plt.close('all')
abcisse = 'length_connection_rat'
ordinate = 'branchin_dist'
tab = corrected
baits_sort = tab.sort_values(abcisse)
N=3
moving_av = baits_sort.rolling(N,min_periods=N//4).mean()
moving_std = baits_sort.rolling(N,min_periods=N//4).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
# ax.set_xlabel('number of trunk hyphae')
ax.set_xlabel('network length/number of trunk hyphae ($\mu m$)')

ax.set_ylabel('Branching distance (BAS and RH undistinguished) ($\mu m$)',color='red')
# ax.set_xlim(0,70)
# ax.set_ylim(0,1500)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
ns = list(set(corrected['n_connect'].values))
ns.sort()
averages =  [np.mean(corrected.loc[corrected["n_connect"]==n][ordinate]) for n in ns]
sts = [np.std(corrected.loc[corrected["n_connect"]==n][ordinate])/np.sqrt(len(corrected.loc[corrected["n_connect"]==n][ordinate])) for n in ns]
# plt.errorbar(ns,averages,yerr=sts, linestyle="None",capthick = 1)
def func2(t,alpha,beta):
    return((beta*np.exp(-alpha*(t))))
popt1, pcov = curve_fit(func2,tab[abcisse],tab[ordinate],bounds = ([0,0],2*[np.inf]),p0=[1,1])
# popt2, pcov = curve_fit(func2, times, total_anastomosis_theory,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])

# ax.plot(ns,func2(np.array(ns),*popt1),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='blue')
ax.scatter(tab[abcisse],tab[ordinate],alpha=0.3,color='red')
# ax.plot(moving_av[abcisse],moving_av[ordinate],color='red',label = 'moving average')
# ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
# ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.spines['right'].set_color('red')
ax.tick_params(axis='y', colors='red')  #setting up Y-axis tick color to black

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.scatter(nconnection,bd)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …